##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Distributed Training in TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/distribute_strategy."><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/guide/distribute_strategy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r2/guide/distribute_strategy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

The `tf.distribute.Strategy` API is an easy way to distribute your training
across multiple devices/machines. Our goal is to allow users to use existing
models and training code with minimal changes to enable distributed training.

Currently, in core TensorFlow, we support `tf.distribute.MirroredStrategy`. This
does in-graph replication with synchronous training on many GPUs on one machine.
Essentially, we create copies of all variables in the model's layers on each
device. We then use all-reduce to combine gradients across the devices before
applying them to the variables to keep them in sync.

Many other strategies will soon be
available in core TensorFlow. You can find more information about them in the
[README](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/distribute).
You can also read the
[public design review](https://github.com/tensorflow/community/blob/master/rfcs/20181016-replicator.md)
for updating the `tf.distribute.Strategy` API as part of the move to
to core TF.



## Example with Keras API

Let's see how to scale to multiple GPUs on one machine using `MirroredStrategy`
with [tf.keras](https://www.tensorflow.org/guide/keras).

We will take a very simple model consisting of a single layer. First, we will import Tensorflow.

In [0]:
from __future__ import absolute_import, division, print_function

In [0]:
!pip install tf-nightly-2.0-preview
import tensorflow as tf

To distribute a Keras model on multiple GPUs using `MirroredStrategy`, we first instantiate a `MirroredStrategy` object.

In [0]:
strategy = tf.distribute.MirroredStrategy()

We then create and compile the Keras model in `strategy.scope`.

In [0]:
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))

Let's also define a simple input dataset for training this model.

In [0]:
train_dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(10000).batch(10)

To train the model we call Keras `fit` API using the input dataset that we
created earlier, same as how we would in a non-distributed case.

In [0]:
model.fit(train_dataset, epochs=5, steps_per_epoch=10)

Similarly, we can also call `evaluate` and `predict` as before using appropriate
datasets.

In [0]:
eval_dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(10)
model.evaluate(eval_dataset, steps=10)


In [0]:
predict_dataset = tf.data.Dataset.from_tensors(([1.])).repeat(10).batch(2)
model.predict(predict_dataset, steps=5)

That's all you need to train your model with Keras on multiple GPUs with
`MirroredStrategy`. It will take care of splitting up
the input dataset, replicating layers and variables on each device, and
combining and applying gradients.

The model and input code does not have to change because we have changed the
underlying components of TensorFlow (such as optimizer, batch norm and
summaries) to become strategy-aware. That means those components know how to
combine their state across devices. Further, saving and checkpointing works
seamlessly, so you can save with one or no distribute strategy and resume with
another.

## Example with Estimator API

You can also use `tf.distribute.Strategy` API with
[`Estimator`](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator).
Let's see a simple example of it's usage with `MirroredStrategy`.

We will use the `LinearRegressor` premade estimator as an example. To use `MirroredStrategy` with Estimator, all we need to do is:

* Create an instance of the `MirroredStrategy` class.
* Pass it to the
[`RunConfig`](https://www.tensorflow.org/api_docs/python/tf/estimator/RunConfig)
parameter of the custom or premade `Estimator`.

In [0]:
strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(
    train_distribute=strategy, eval_distribute=strategy)
regressor = tf.estimator.LinearRegressor(
    feature_columns=[tf.feature_column.numeric_column('feats')],
    optimizer='SGD',
    config=config)

We will define a simple input function to feed data for training this model.

In [0]:
def input_fn():
  return tf.data.Dataset.from_tensors(({"feats":[1.]}, [1.])).repeat(10000).batch(10)

Then we can call `train` on the regressor instance to train the model.

In [0]:
regressor.train(input_fn=input_fn, steps=10)


And we can `evaluate` to evaluate the trained model.

In [0]:
regressor.evaluate(input_fn=input_fn, steps=10)

That's it! This change will now configure estimator to run on all GPUs on your
machine.


## Customization and Performance Tips

Above, we showed the easiest way to use `MirroredStrategy`. There are few things
you can customize in practice:

*   You can specify a list of specific GPUs (using param `devices`), in case you
    don't want auto detection.
*   You can specify various parameters for all reduce with the
    `cross_device_ops` param, such as the all reduce algorithm to use, and
    gradient repacking.

We've tried to make it such that you get the best performance for your existing
model without having to specify any additional options. We also recommend you
follow the tips from
[Input Pipeline Performance Guide](https://www.tensorflow.org/performance/datasets_performance).
Specifically, we found using
[`map_and_batch`](https://www.tensorflow.org/performance/datasets_performance#map_and_batch)
and
[`dataset.prefetch`](https://www.tensorflow.org/performance/datasets_performance#pipelining)
in the input function gives a solid boost in performance. When using
`dataset.prefetch`, use `buffer_size=tf.data.experemental.AUTOTUNE` to let it detect optimal buffer size.

## Caveats

This API is still in progress there are a lot of improvements forthcoming:

*   Summaries are only computed in the first replica in `MirroredStrategy`.
*   PartitionedVariables are not supported yet.
*   Performance improvements, especially w.r.t. input handling, eager execution
    etc.

## What's next?

`tf.distribute.Strategy` is actively under development and we will be adding more examples and tutorials in the near future. Please give it a try, we welcome your feedback via [issues on GitHub](https://github.com/tensorflow/tensorflow/issues/new).